# Funções para recomendações

In [62]:
from math import sqrt

# Dataframe fictício para testes simples
from recomendacoes import *

# Dataframe real fornecido pelo MovieLens
def df_movie_lens(path='ml-100k'):
    # Inicializa dicionário de filmes
    filmes = {}
    for linha in open(path + '/u.item', encoding='latin-1'):
        campos = linha.split('|')
        id_filme, titulo = campos[0], campos[1]
        filmes[id_filme] = titulo

    base = {}
    # Percorre os usuários e armazena no dicionário
    for linha in open(path + '/u.data', encoding='latin-1'):
        usuario, id_filme, nota, tempo = linha.split('\t')
        base.setdefault(usuario, {})
        base[usuario][id_filme] = float(nota)

    return base, filmes

base_movie_lens, filmes = df_movie_lens('ml-100k')

def recomenda_itens(base_usuario, similaridade_itens, usuario, id_para_titulo):
    notasUsuario = base_usuario[usuario]
    notas = {}
    totalSimilaridade = {}

    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridade_itens[item]:
            if item2 in notasUsuario:
                continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2, 0)
            totalSimilaridade[item2] += similaridade

    rankings = [(score / totalSimilaridade[item], id_para_titulo[item]) for item, score in notas.items()]
    rankings.sort()
    rankings.reverse()

    # Adicione instruções de impressão para depuração
    print(f"Notas: {notas}")
    print(f"Total de Similaridade: {totalSimilaridade}")

    return rankings


def euclidiana(base, usuario1, usuario2):
    si = {item: 1 for item in base[usuario1] if item in base[usuario2]}

    if len(si) == 0:
        return 0  # Nenhuma similaridade se não houver itens em comum

    soma = sum([pow(base[usuario1][item] - base[usuario2][item], 2) for item in si])
    return 1 / (1 + sqrt(soma))


def users_similares(base, usuario):
    similaridade = [(euclidiana(base, usuario, outro), outro) for outro in base if outro != usuario]
    similaridade.sort()
    similaridade.reverse()
    return similaridade[0:10]

def recomenda_users(base, usuario):
    totais = {}
    somaSimilaridade = {}

    for outro in base:
        if outro == usuario:
            continue
        similaridade = euclidiana(base, usuario, outro)

        if similaridade <= 0:
            continue

        for item in base[outro]:
            if item not in base[usuario]:
                totais.setdefault(item, 0)
                totais[item] += base[outro][item] * similaridade
                somaSimilaridade.setdefault(item, 0)
                somaSimilaridade[item] += similaridade

    rankings = [(total / somaSimilaridade[item], item) for item, total in totais.items()]
    rankings.sort()
    rankings.reverse()
    return rankings[0:30]

def calcula_itens_similares(base):
    result = {}
    for item in base:
        notas = users_similares(base, item)
        result[item] = notas
    return result


def recomenda_itens(base, similaridade_itens, usuario, id_para_titulo):
    notasUsuario = base[usuario]
    notas = {}
    totalSimilaridade = {}

    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridade_itens[item]:
            if item2 in notasUsuario:
                continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2, 0)
            totalSimilaridade[item2] += similaridade

    rankings = [(score / totalSimilaridade[item], id_para_titulo[item]) for item, score in notas.items()]
    rankings.sort()
    rankings.reverse()

    # Adicione instruções de impressão para depuração
    print(f"Notas: {notas}")
    print(f"Total de Similaridade: {totalSimilaridade}")

    return rankings

def recomenda_itens_ficticio(base, similaridade_itens, usuario):
    notasUsuario = base[usuario]
    notas = {}
    totalSimilaridade = {}

    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridade_itens[item]:
            if item2 in notasUsuario:
                continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2, 0)
            totalSimilaridade[item2] += similaridade

    rankings = [(score / totalSimilaridade[item]) for item, score in notas.items()]
    rankings.sort()
    rankings.reverse()

    # Adicione instruções de impressão para depuração
    print(f"Notas: {notas}")
    print(f"Total de Similaridade: {totalSimilaridade}")

    return rankings

# Testes

### Base de dados fictícia

In [63]:
recomenda_users(avaliacoes_usuarios, 'Leonardo')

[(3.457128694491423, 'Star Wars'),
 (2.778584003814924, 'Freddy x Jason'),
 (2.422482042361917, 'Star Trek')]

In [64]:
itens_similares = calcula_itens_similares(avaliacoes_filmes)

print(itens_similares)

{'Freddy x Jason': [(0.4721359549995794, 'Norbit'), (0.3761785115301142, 'Star Wars'), (0.3567891723253309, 'O Ultimato Bourne'), (0.3483314773547883, 'Star Trek'), (0.2402530733520421, 'Exterminador do Futuro')], 'O Ultimato Bourne': [(0.3761785115301142, 'Exterminador do Futuro'), (0.3567891723253309, 'Freddy x Jason'), (0.3266316347104093, 'Star Wars'), (0.32037724101704074, 'Star Trek'), (0.1886378647726465, 'Norbit')], 'Star Trek': [(0.3761785115301142, 'Norbit'), (0.3483314773547883, 'Freddy x Jason'), (0.32037724101704074, 'O Ultimato Bourne'), (0.2708131845707603, 'Star Wars'), (0.20799159651347807, 'Exterminador do Futuro')], 'Exterminador do Futuro': [(0.3761785115301142, 'O Ultimato Bourne'), (0.27429188517743175, 'Star Wars'), (0.2402530733520421, 'Freddy x Jason'), (0.20799159651347807, 'Star Trek'), (0.18464218557642828, 'Norbit')], 'Norbit': [(0.4721359549995794, 'Freddy x Jason'), (0.3761785115301142, 'Star Trek'), (0.29429805508554946, 'Star Wars'), (0.1886378647726465

In [66]:
recomenda_itens_ficticio(avaliacoes_usuarios, itens_similares, 'Leonardo')

Notas: {'Freddy x Jason': 3.0386995238717365, 'Star Wars': 2.861307951992118, 'Star Trek': 2.6498424821607096}
Total de Similaridade: {'Freddy x Jason': 1.0691782006769524, 'Star Wars': 0.8952215749733905, 'Star Trek': 0.904547349060633}


[3.1962008423190285, 2.929467965289551, 2.8420889258196413]

# Testes
## Base de dados MovieLens

In [59]:
# Exemplo 1: Encontrar usuários similares a um usuário específico
usuario_exemplo1 = '1'
usuarios_similares_exemplo1 = users_similares(base_movie_lens, usuario_exemplo1)
print(f"Usuários similares ao usuário {usuario_exemplo1}:")
for similaridade, outro_usuario in usuarios_similares_exemplo1:
    print(f"Usuário {outro_usuario} - Similaridade: {similaridade}")

Usuários similares ao usuário 1:
Usuário 812 - Similaridade: 1.0
Usuário 418 - Similaridade: 1.0
Usuário 155 - Similaridade: 1.0
Usuário 729 - Similaridade: 0.5
Usuário 631 - Similaridade: 0.5
Usuário 351 - Similaridade: 0.5
Usuário 309 - Similaridade: 0.5
Usuário 273 - Similaridade: 0.5
Usuário 876 - Similaridade: 0.4142135623730951
Usuário 485 - Similaridade: 0.4142135623730951


In [67]:
# Exemplo 2: Fornecer recomendações de filmes para um usuário específico
usuario_exemplo2 = '1'
recomendacoes_exemplo2 = recomenda_users(base_movie_lens, usuario_exemplo2)
print(f"Recomendações de filmes para o usuário {usuario_exemplo2}:")
for pontuacao, filme in recomendacoes_exemplo2:
    print(f"Filme: {filme} - Pontuação: {pontuacao}")


Recomendações de filmes para o usuário 1:
Filme: 1467 - Pontuação: 5.000000000000001
Filme: 1189 - Pontuação: 5.000000000000001
Filme: 814 - Pontuação: 5.0
Filme: 1653 - Pontuação: 5.0
Filme: 1599 - Pontuação: 5.0
Filme: 1536 - Pontuação: 5.0
Filme: 1500 - Pontuação: 5.0
Filme: 1201 - Pontuação: 5.0
Filme: 1122 - Pontuação: 5.0
Filme: 1293 - Pontuação: 4.999999999999999
Filme: 1449 - Pontuação: 4.709540786352371
Filme: 1642 - Pontuação: 4.624759150439772
Filme: 1398 - Pontuação: 4.605117943969987
Filme: 1191 - Pontuação: 4.551685512163036
Filme: 408 - Pontuação: 4.512901471394128
Filme: 318 - Pontuação: 4.503322988079016
Filme: 483 - Pontuação: 4.474105171116072
Filme: 513 - Pontuação: 4.448398214360137
Filme: 1367 - Pontuação: 4.4388914364740515
Filme: 1594 - Pontuação: 4.4121968395559765
Filme: 603 - Pontuação: 4.403246514259
Filme: 1639 - Pontuação: 4.365085124965483
Filme: 285 - Pontuação: 4.332368408305184
Filme: 1396 - Pontuação: 4.331407117669634
Filme: 480 - Pontuação: 4.326677

In [68]:
# Exemplo 3: Calcular itens similares e fornecer recomendações de itens para um usuário
itens_similares = calcula_itens_similares(base_movie_lens)
usuario_exemplo3 = '1'
recomendacoes_itens_exemplo3 = recomenda_itens(base_movie_lens, itens_similares, usuario_exemplo3, filmes)
print(f"Recomendações de itens para o usuário {usuario_exemplo3}:")
for pontuacao, item in recomendacoes_itens_exemplo3:
    print(f"Item: {item} - Pontuação: {pontuacao}")



Notas: {'939': 81.0, '891': 34.0, '852': 23.65685424949238, '815': 28.0, '766': 34.5, '705': 5.0, '812': 46.0, '431': 18.598076211353316, '888': 59.79674482879475, '866': 43.5, '811': 57.985281374238575, '935': 36.5, '903': 13.0, '901': 24.0, '892': 10.0, '861': 38.41421356237309, '859': 39.32842712474619, '814': 53.572767706041475, '798': 15.0, '767': 67.98698126841458, '856': 65.0, '813': 60.5, '784': 31.5, '720': 52.15685424949238, '688': 70.57106781186548, '628': 18.869693845669907, '410': 17.0, '302': 20.55492646310117, '905': 20.5, '787': 25.0, '755': 24.0, '702': 17.0, '681': 21.0, '662': 18.413719988157784, '448': 27.0, '915': 43.0, '923': 13.0, '917': 48.0, '872': 14.0, '789': 29.5, '714': 10.0, '644': 5.0, '595': 9.0, '941': 57.098076211353316, '922': 22.0, '900': 29.0, '868': 28.0, '759': 5.0, '726': 36.66914402321879, '723': 17.89564392373896, '716': 20.0, '558': 45.96796292722099, '522': 59.480375165651424, '473': 12.0, '366': 16.045084971874736, '358': 16.0, '934': 2.0, '